# Sensitivity analysis on previous work 

Training and testing data from all
actions and test subjects, but 0 degrees aspect angle.

In [1]:
# Needed to allow editing using PyCharm
%load_ext autoreload
%autoreload 2

The following cell is needed for compatibility when using both CoLab and Local Jupyter notebook.
It sets the appropriate file path for the data and also installs local packages such as models.

In [ ]:
import os
if os.getcwd() == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    FILE_PATH = '/content/gdrive/My Drive/Level-4-Project/data/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
else:
    FILE_PATH = "C:/Users/macka/Google Drive/Level-4-Project/data/"
    
from src.models.original_models import cifar_initialised

In [2]:
import numpy as np
import sys
from six.moves import cPickle
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
import sys
from sklearn.metrics import classification_report, confusion_matrix
import csv
from keras.models import load_model
from src.models.original_models import cifar_initialised

Using TensorFlow backend.


In [ ]:
# Needed as originally code was for theano backend but now using tensor flow
from keras import backend as K
K.set_image_dim_ordering('th')

In [5]:
def load_batch(fpath, label_key='labels'):
    f = open(fpath, 'rb')
    if sys.version_info < (3,):
        d = cPickle.load(f)
    else:
        d = cPickle.load(f, encoding="bytes")
        # decode utf8
        for k, v in d.items():
            # added check as otherwise tries to decode a string
            if type(k) is not str:
                del(d[k])
                d[k.decode("utf8")] = v
    f.close()
    data = d["data"]
    labels = d[label_key]

    data = data.reshape(data.shape[0], 1, 75, 75)
    return data, labels

In [6]:
def load_data(data_folder):
    path = FILE_PATH + data_folder

    nb_train_samples = 35595

    X_train = np.zeros((nb_train_samples, 1, 75, 75), dtype="uint8")
    y_train = np.zeros((nb_train_samples,), dtype="uint8")

    for i in range(1, 6):
        fpath = os.path.join(path, 'data_batch_' + str(i))
        data, labels = load_batch(fpath)
        X_train[(i-1)*7119:i*7119, :, :, :] = data
        y_train[(i-1)*7119:i*7119] = labels

    fpath = os.path.join(path, 'test_batch')
    X_test, y_test = load_batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    return (X_train, y_train), (X_test, y_test)

In [14]:
batch_size = 100
nb_classes = 7
nb_epoch = 100
nb_epoch = 2 # originally 100
data_augmentation = False

# input image dimensions
img_rows, img_cols = 75, 75
# the CIFAR10 images are RGB


# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = load_data('cifar_initialised')
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (35595, 1, 75, 75)
35595 train samples
7119 test samples


In [15]:
models = []
for version_number in range(1,5):
    models.append(cifar_initialised.make_model(version_number, img_rows, img_cols, nb_classes))

In [16]:
for model in models:
    # let's train the model using SGD + momentum (how original).
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255 
    if not data_augmentation:
        print('Not using data augmentation.')
        hist = model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  epochs=nb_epoch,
                  validation_data=(X_test, Y_test),
                  shuffle=True)

        y_pred = model.predict_classes(X_test)
        print(y_pred)
        target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting', 'Walking']
#         sys.stdout = open('/home/aleksandar/sets/cifar_initialised/results/report_initialised.txt', "w")
        print("CLASSIFICATION REPORT:")
        print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
        print ("\n")
        print("CONFUSION MATRIX:")
        print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))

#         with open('/home/aleksandar/sets/cifar_initialised/results/graphs_initialised.csv', "w") as f:
#            w = csv.writer(f)
#            train_loss = hist.history['loss']
#            val_loss = hist.history['val_loss']
#            train_acc = hist.history['acc']
#            val_acc = hist.history['val_acc']
#            helplist = list(zip(train_loss, train_acc, val_loss, val_acc))
#            w.writerow("train_loss, train_acc, val_loss, val_acc")
#            for val in helplist:
#               w.writerow([val])


#         model.save('/home/aleksandar/sets/cifar_initialised/results/model_initialised.h5')
#         #-----------------------

    else:
        print('Using real-time data augmentation.')

        # this will do preprocessing and realtime data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images

        # compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied)
        datagen.fit(X_train)

        # fit the model on the batches generated by datagen.flow()
        hist = model.fit_generator(datagen.flow(X_train, Y_train,
                            batch_size=batch_size),
                            samples_per_epoch=X_train.shape[0],
                            nb_epoch=nb_epoch,
                            validation_data=(X_test, Y_test))

        #NEW--------------------
        y_pred = model.predict_classes(X_test)
        print(y_pred)
        target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting', 'Walking']
#         sys.stdout = open('/home/aleksandar/sets/cifar_initialised/results/report_initialised.txt', "w")
        print("CLASSIFICATION REPORT:")
        print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
        print ("\n")
        print("CONFUSION MATRIX:")
        print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))

#         with open('/home/aleksandar/sets/cifar_initialised/results/graphs_initialised.csv', "w") as f:
#            w = csv.writer(f)
#            train_loss = hist.history['loss']
#            val_loss = hist.history['val_loss']
#            train_acc = hist.history['acc']
#            val_acc = hist.history['val_acc']
#            helplist = list(zip(train_loss, train_acc, val_loss, val_acc))
#            w.writerow("train_loss, train_acc, val_loss, val_acc")
#            for val in helplist:
#               w.writerow([val])


#         model.save('/home/aleksandar/sets/cifar_initialised/results/model_initialised.h5')
#         #-----------------------

Not using data augmentation.
Train on 35595 samples, validate on 7119 samples
Epoch 1/2
35595/35595 [==============================] - 93s 3ms/step - loss: 0.5997 - acc: 0.7701 - val_loss: 0.0749 - val_acc: 0.9781
Epoch 2/2
35595/35595 [==============================] - 82s 2ms/step - loss: 0.0997 - acc: 0.9667 - val_loss: 0.0606 - val_acc: 0.9871
[6 4 4 ... 5 1 3]
CLASSIFICATION REPORT:
                  precision    recall  f1-score   support

ArmFasterTowards       0.98      0.99      0.99      1017
ArmSlowerTowards       1.00      0.98      0.99      1017
     CirclingArm       0.99      0.95      0.97      1017
        Clapping       0.95      1.00      0.97      1017
       PickingUp       1.00      0.99      1.00      1017
         Sitting       1.00      1.00      1.00      1017
         Walking       1.00      1.00      1.00      1017

       micro avg       0.99      0.99      0.99      7119
       macro avg       0.99      0.99      0.99      7119
    weighted avg       0.99

C:\Users\macka\Anaconda3\envs\dataScience\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Not using data augmentation.
Train on 35595 samples, validate on 7119 samples
Epoch 1/2
35595/35595 [==============================] - 42s 1ms/step - loss: 1.9462 - acc: 0.1385 - val_loss: 1.9460 - val_acc: 0.1429
Epoch 2/2
35595/35595 [==============================] - 42s 1ms/step - loss: 1.9462 - acc: 0.1385 - val_loss: 1.9460 - val_acc: 0.1429
[2 2 2 ... 2 2 2]
CLASSIFICATION REPORT:
                  precision    recall  f1-score   support

ArmFasterTowards       0.00      0.00      0.00      1017
ArmSlowerTowards       0.00      0.00      0.00      1017
     CirclingArm       0.14      1.00      0.25      1017
        Clapping       0.00      0.00      0.00      1017
       PickingUp       0.00      0.00      0.00      1017
         Sitting       0.00      0.00      0.00      1017
         Walking       0.00      0.00      0.00      1017

       micro avg       0.14      0.14      0.14      7119
       macro avg       0.02      0.14      0.04      7119
    weighted avg       0.02